In [1]:
import os
from urllib.parse import urlparse
import mlflow
import numpy as np
import pandas as pd
import dask.dataframe as dd
from config import settings

In [2]:
from helpers.kaggle_submission import kaggle_submission
from process_and_ml.pipeline import preprocessing_baseline, show_metrics_baseline


In [3]:
from catboost import CatBoostClassifier, Pool

In [4]:
from catboost.utils import get_gpu_device_count
from process_and_ml.feature_engineer import fs_for_cat_boost
print('I see %i GPU devices' % get_gpu_device_count())

I see 1 GPU devices


In [5]:
import sys
sys.path.append("..")
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs


In [6]:
from config import settings

In [11]:
where_is = WhereIs(False)

In [8]:
all_paths = where_is.get_paths_list

In [9]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0])


In [10]:
customers_df = load_customers(all_paths[1])

In [14]:
test_file, lnr = load_test_file(all_paths[6])

KeyError: 'LNR'

In [15]:
clean_up = CleanUp(all_paths)

In [16]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df, 'test_file': test_file}

NameError: name 'customers_df' is not defined

In [17]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_dict = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[5], info_df=all_paths[9],attr_df=all_paths[8])

NameError: name 'argument_dict' is not defined

In [10]:
path_customer = '../data/cat_dfs/cleaned_customers.csv'
path_azdias = '../data/cat_dfs/cleaned_azdias.csv'

In [12]:
dfs_dict['azdias'].to_csv('../data/cat_dfs/cleaned_azdias.csv', index=False)
dfs_dict['customers_df'].to_csv('../data/cat_dfs/cleaned_customers.csv', index=False)

NameError: name 'dfs_dict' is not defined

In [19]:
# run with dask

## Run the same algo with entire dataset
#df_to_model = fs_for_cat_boost(df_population=dfs_dict['azdias'], df_customer=dfs_dict['customers_df'])
df_to_model = dd.concat(
        [dd.read_csv(path_customer).assign(is_customer=1),
         dd.read_csv(path_azdias).assign(is_customer=0)]
    )

In [20]:
len(df_to_model)

1082873

In [21]:
path = '../data/cat_dfs/cleaned_customers_populations.csv'

In [22]:

df_to_model.to_csv(path, single_file=True)


['/home/jair/Documents/arvato_project/data/cat_dfs/cleaned_customers_populations.csv']

In [ ]:
dataframe_concatenated = pd.read_csv(path, index_col=[0])


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
dataframe_concatenated.head()

In [ ]:
#dataframe_concatenated = dataframe_concatenated.loc[:, ~dataframe_concatenated.columns.str.match('Unnamed')]

In [ ]:
df_to_model = pd.concat(
    [dfs_dict['customers_df'].assign(is_customer=1),
     dfs_dict['azdias'].assign(is_customer=0)]
)

In [ ]:
## Fazendo o dataframe todo com Dask


In [ ]:
cat_features = dataframe_concatenated.select_dtypes(include=['category', 'object']).columns

In [ ]:
cat_features

In [ ]:
features, labels = preprocessing_baseline(dataframe_concatenated,
                                          cat_features=cat_features,
                                          target='is_customer')

In [ ]:
X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

In [ ]:
params_catboost = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=5,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40)

In [ ]:
params_catboost_2 = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=20,
                              one_hot_max_size=4,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC')

### Mlflow data

In [ ]:
tags = {'algo':'catboost',
        'datasets':'population_customers',
        'sample':1,
        'cols':cat_features,
        'rows':dataframe_concatenated.shape[0]}
run_name = 'first_cat'


In [ ]:
from pathlib import Path
path = (
    Path('.') if Path('.').resolve().name == 'arvato_project'
    else Path('..')
)

In [ ]:
path

In [ ]:
path_mlflow = path / 'mlruns'
mlflow.set_tracking_uri(str(path_mlflow))
#experiment_id = mlflow.create_experiment('CatBoostClassifier')



In [ ]:
mlflow.get_tracking_uri()

## Set new experiment

In [ ]:
mlflow.set_experiment(experiment_name='CatBoostClassifier')


## Defining classificator


In [ ]:
catboost = CatBoostClassifier(**params_catboost_2)

##

In [ ]:
with mlflow.start_run(tags=tags):
    mlflow.log_params(params_catboost_2)
    catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    metrics_returned = show_metrics_baseline(catboost, features=features, labels=labels)
    mlflow.log_metrics(metrics_returned)

In [12]:
catboost.feature_names_

NameError: name 'catboost' is not defined

In [55]:
params_catboost = dict(num_trees=40,
                              task_type='GPU',
                              max_depth=6,
                              min_child_samples=40,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40,
                              cv='Inverted')


## Começa aqui a rodar para a primeira submissao no Kaggle

In [21]:
from process_and_ml.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool

In [29]:
#Run with just 2 columns
population = pd.read_csv('../data/cleaned_data/azdias_clean.csv', index_col=[0]).sample(frac=0.4, random_state=settings.RANDOM_STATE)
customers = pd.read_csv('../data/cleaned_data/customers_clean.csv', index_col=[0]).sample(frac=0.4, random_state=settings.RANDOM_STATE)

In [30]:
df_to_model = pd.concat(
    [customers.assign(is_customer=1),
     population.assign(is_customer=0)
]
)

In [32]:
cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

In [33]:
cat_features


Index(['CAMEO_DEU_2015', 'OST_WEST_KZ'], dtype='object')

In [34]:
features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')

X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

In [51]:
tags = {'algo':'catboost',
        'datasets':'population_customers',
        'sample':0.4,
        'cols':cat_features,
        'rows':df_to_model.shape[0]}
run_name = 'first_cat'


In [36]:
params_catboost_3 = dict(num_trees=30,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=20,
                              one_hot_max_size=4,
                              cat_features=cat_features,
                              random_state=settings.RANDOM_STATE,
                              eval_metric='AUC')


In [52]:
catboost = CatBoostClassifier(**params_catboost)

In [56]:
with mlflow.start_run(tags=tags):
    mlflow.log_params(params_catboost)
    catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    metrics_returned = show_metrics_baseline(catboost, features=features, labels=labels)
    mlflow.log_metrics(metrics_returned)

Accuracy Train: 0.7994122768828907
AUC Train: 0.8957117187100356
Accuracy Test: 0.797051037369944
AUC Test: 0.8926755665241357
Accuracy Valid: 0.7989164729964755
AUC Valid: 0.8946125057266858


In [2]:
import pandas as pd
path_test_clean = '../data/cleaned_data/test_correct.csv'
test_file= pd.read_csv(path_test_clean)
lnr = test_file['LNR']
test_file.drop(['LNR'], axis=1, inplace=True)

In [3]:
from process_and_ml.pipeline import cat_features_fill_na

test_df_filled = cat_features_fill_na(test_file, cat_features=['CAMEO_DEU_2015', 'OST_WEST_KZ'])

In [4]:
## kaggle submission

from helpers.kaggle_submission import kaggle_submission

### LOAD MODEL


In [22]:
from process_and_ml.pipeline import load_catboost_model

catboost = load_catboost_model('catboost_first.cbm')

In [9]:
catboost

In [26]:
y_pred=catboost.predict_proba(test_df_filled)[:, 1]

In [54]:

kaggle_submission(
    column_lnr=lnr,
    y_pred=catboost.predict_proba(test_df_filled)[:, 1],
    submission_filename='catboost_third_shoot',
    submission_message='CatBoost trained [20trees] with a sample of a 40% of the data [70/15/15], checking result')



Successfully submitted to Udacity+Arvato: Identify Customer Segments


100%|██████████| 1.02M/1.02M [00:04<00:00, 252kB/s] 


In [31]:
# After a poor prediction let's check the train file



W          26773
O           8433
UNKNOWN     7627
Name: OST_WEST_KZ, dtype: int64

In [28]:
from process_and_ml.pipeline import save_catboost_model
save_catboost_model(catboost_model=catboost, model_name='catboost_first.cbm', pool_data=Pool(X_train, y_train, cat_features=cat_features))


